In [0]:
%sql
-- 1. Cria o banco de dados para a camada Curated (Otimizada para análise)
CREATE DATABASE IF NOT EXISTS curated_data;

-- 2. Cria a Tabela de Dimensão 'dim_ocupacao'
CREATE OR REPLACE TABLE curated_data.dim_ocupacao
USING DELTA
AS
SELECT DISTINCT
    ROW_NUMBER() OVER (ORDER BY Job_Title) AS id_ocupacao, -- Cria a chave substituta (PK)
    Job_Title,
    Education_Level,
    Risk_Category
FROM default.ai_impact_on_jobs_2030; -- Substitua 'default.ai_impact_on_jobs_2030' pelo nome exato da sua tabela RAW
-- 1. Verifique o conteúdo da nova tabela Dimensão
SELECT * FROM curated_data.dim_ocupacao LIMIT 10;

-- 2. Verifique quantas linhas únicas foram criadas
SELECT count(*) FROM curated_data.dim_ocupacao;

In [0]:
%sql
-- 1. CRIAR a Tabela Fato 'fato_risco_automacao'
-- Usamos o ROW_NUMBER() para garantir a mesma ordem da Dimensão, garantindo a unicidade.
CREATE OR REPLACE TABLE curated_data.fato_risco_automacao
USING DELTA
AS
SELECT
    d.id_ocupacao, -- Garante o ID único da ocupação
    t.Average_Salary,
    t.Years_Experience,
    t.AI_Exposure_Index,
    t.Tech_Growth_Factor,
    t.Automation_Probability_2030,
    t.Skill_1, t.Skill_2, t.Skill_3, t.Skill_4, t.Skill_5,
    t.Skill_6, t.Skill_7, t.Skill_8, t.Skill_9, t.Skill_10
FROM default.ai_impact_on_jobs_2030 t -- Tabela RAW
INNER JOIN (
    SELECT Job_Title, ROW_NUMBER() OVER (ORDER BY Job_Title) AS id_ocupacao
    FROM (SELECT DISTINCT Job_Title FROM default.ai_impact_on_jobs_2030) -- Subconsulta para garantir a unicidade
) d
  ON t.Job_Title = d.Job_Title;

-- 2. NOVA VERIFICAÇÃO: Conte o número de linhas na Tabela Fato.
SELECT count(*) FROM curated_data.fato_risco_automacao;

In [0]:
%sql
-- Pergunta 1: Top 10 Ocupações com Maior Risco Médio de Automação
SELECT
    d.Job_Title,
    AVG(f.Automation_Probability_2030) AS Risco_Medio_Automacao
FROM curated_data.fato_risco_automacao f
JOIN curated_data.dim_ocupacao d
  ON f.id_ocupacao = d.id_ocupacao
GROUP BY 1
ORDER BY Risco_Medio_Automacao DESC
LIMIT 10;

In [0]:
%sql
-- Pergunta 2: Correlação entre Salário e Risco de Automação
-- Nota: Usamos as colunas da Fato, pois é a camada curada.
SELECT
    CORR(Average_Salary, Automation_Probability_2030) AS Correlacao_Salario_Risco
FROM curated_data.fato_risco_automacao;

In [0]:
%sql
-- Pergunta 3: Correlação entre Habilidades e Exposição à IA
-- Usamos uma única consulta para correlacionar o Índice de Exposição à IA (AI_Exposure_Index)
-- com cada uma das 10 habilidades.

SELECT
    'Skill_1' AS Habilidade, CORR(AI_Exposure_Index, Skill_1) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_2' AS Habilidade, CORR(AI_Exposure_Index, Skill_2) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_3' AS Habilidade, CORR(AI_Exposure_Index, Skill_3) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_4' AS Habilidade, CORR(AI_Exposure_Index, Skill_4) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_5' AS Habilidade, CORR(AI_Exposure_Index, Skill_5) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_6' AS Habilidade, CORR(AI_Exposure_Index, Skill_6) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_7' AS Habilidade, CORR(AI_Exposure_Index, Skill_7) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_8' AS Habilidade, CORR(AI_Exposure_Index, Skill_8) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_9' AS Habilidade, CORR(AI_Exposure_Index, Skill_9) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
UNION ALL
SELECT
    'Skill_10' AS Habilidade, CORR(AI_Exposure_Index, Skill_10) AS Correlacao_Exposicao_IA
FROM curated_data.fato_risco_automacao
ORDER BY Correlacao_Exposicao_IA DESC;

In [0]:
%sql
-- Encontra as 10 ocupações com a maior probabilidade média de automação
SELECT
    d.Job_Title,
    d.Education_Level,
    d.Risk_Category,
    AVG(f.Automation_Probability_2030) AS Avg_Automation_Risk,
    COUNT(f.id_ocupacao) AS Total_Records
FROM curated_data.fato_risco_automacao f
JOIN curated_data.dim_ocupacao d
  ON f.id_ocupacao = d.id_ocupacao
GROUP BY 1, 2, 3
ORDER BY Avg_Automation_Risk DESC
LIMIT 10;

In [0]:
#%python
# 1. Importa bibliotecas necessárias
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 2. Transfere o resultado da última query SQL (_sqldf) para um DataFrame Pandas
# Usamos df_top_risco como o nome da variável, substituindo 'top_risco' não definida.
df_top_risco = spark.sql("""
    SELECT
        d.Job_Title,
        AVG(f.Automation_Probability_2030) AS Avg_Automation_Risk
    FROM curated_data.fato_risco_automacao f
    JOIN curated_data.dim_ocupacao d
      ON f.id_ocupacao = d.id_ocupacao
    GROUP BY 1
    ORDER BY Avg_Automation_Risk DESC
    LIMIT 10
""").toPandas()

# Se você usou o %sql na célula anterior, e o resultado foi _sqldf, pode ser mais simples:
# df_top_risco = _sqldf.toPandas()

In [0]:
%python
# 3. Cria a visualização (Gráfico de Barras Horizontal)
plt.figure(figsize=(10, 6))

# Usamos df_top_risco no parâmetro 'data'
sns.barplot(x='Avg_Automation_Risk', y='Job_Title', data=df_top_risco, palette='viridis')

plt.title('Top 10 Ocupações com Maior Risco de Automação (Média)')
plt.xlabel('Risco de Automação Médio (0 a 1)')
plt.ylabel('Ocupação')
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

In [0]:
%python
# 1. Carrega as colunas necessárias para o gráfico de dispersão diretamente da Tabela Fato
df_correlacao = spark.sql("""
    SELECT
        Average_Salary,
        Automation_Probability_2030
    FROM curated_data.fato_risco_automacao
""").toPandas()

In [0]:
%python
# 2. Cria o Gráfico de Dispersão com Linha de Regressão (regplot)
plt.figure(figsize=(10, 6))

# O regplot do seaborn já calcula e plota a linha de tendência (regressão)
sns.regplot(x='Average_Salary', y='Automation_Probability_2030', data=df_correlacao, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})

plt.title('Correlação Salário Médio vs. Risco de Automação (CORR ≈ -0.013)')
plt.xlabel('Salário Médio Anual (USD)')
plt.ylabel('Probabilidade de Automação (0 a 1)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [0]:
%python
# 1. Carrega o resultado da última query SQL (Correlação de Habilidades)
# Agora, garantimos que a tabela 'curated_data.fato_risco_automacao' seja especificada para cada SELECT.

df_habilidades = spark.sql("""
    SELECT 'Skill_4' AS Habilidade, CORR(AI_Exposure_Index, Skill_4) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_2' AS Habilidade, CORR(AI_Exposure_Index, Skill_2) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_7' AS Habilidade, CORR(AI_Exposure_Index, Skill_7) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_6' AS Habilidade, CORR(AI_Exposure_Index, Skill_6) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_10' AS Habilidade, CORR(AI_Exposure_Index, Skill_10) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_9' AS Habilidade, CORR(AI_Exposure_Index, Skill_9) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_1' AS Habilidade, CORR(AI_Exposure_Index, Skill_1) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_8' AS Habilidade, CORR(AI_Exposure_Index, Skill_8) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_5' AS Habilidade, CORR(AI_Exposure_Index, Skill_5) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    UNION ALL
    SELECT 'Skill_3' AS Habilidade, CORR(AI_Exposure_Index, Skill_3) AS Correlacao_Exposicao_IA FROM curated_data.fato_risco_automacao
    ORDER BY Correlacao_Exposicao_IA DESC
""").toPandas()

In [0]:
%python
# 2. Cria o Gráfico de Barras para as Correlações de Habilidade (Visualização 3)

import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))

# Usa o DataFrame df_habilidades carregado na Célula 6
sns.barplot(x='Habilidade', y='Correlacao_Exposicao_IA', data=df_habilidades, palette='coolwarm')

plt.title('Correlação entre Habilidades e Índice de Exposição à IA (2030)')
plt.xlabel('Habilidade')
plt.ylabel('Coeficiente de Correlação (r)')
# Linha em y=0 para mostrar que a correlação é mínima
plt.axhline(0, color='grey', linestyle='--')
plt.ylim(-0.03, 0.03) # Limita o eixo Y para enfatizar a proximidade de zero
plt.show()

In [0]:
# Carrega a tabela RAW
df_raw = spark.table("default.ai_impact_on_jobs_2030")

In [0]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Define a janela de particionamento e ordenação para gerar a chave substituta
# Ordenamos por Job_Title para garantir que a ordem seja consistente
window_spec = Window.orderBy("Job_Title")

# 1. Seleciona colunas de Dimensão e remove duplicatas (DISTINCT)
df_dim_ocupacao = df_raw.select("Job_Title", "Education_Level", "Risk_Category").distinct()

# 2. Gera a Chave Primária (PK) 'id_ocupacao'
df_dim_ocupacao = df_dim_ocupacao.withColumn(
    "id_ocupacao",
    row_number().over(window_spec)
)

# 3. Seleciona as colunas finais na ordem correta
df_dim_ocupacao = df_dim_ocupacao.select("id_ocupacao", "Job_Title", "Education_Level", "Risk_Category")

# 4. Salva no Delta Lake na camada Curated
df_dim_ocupacao.write.format("delta").mode("overwrite").saveAsTable("curated_data.dim_ocupacao")

print("Tabela dim_ocupacao criada com sucesso!")
# df_dim_ocupacao.display() # Use display() para ver as primeiras linhas no Databricks

In [0]:
from pyspark.sql.functions import col

# 1. Carrega a dimensão criada para uso no JOIN
df_dim_lookup = spark.table("curated_data.dim_ocupacao")

# 2. Seleciona apenas as chaves de negócio e a PK da Dimensão para o JOIN
df_dim_lookup = df_dim_lookup.select("Job_Title", "id_ocupacao")

# 3. Realiza o JOIN da RAW com a Dimensão para adicionar a FK (id_ocupacao)
df_fato_join = df_raw.join(
    df_dim_lookup,
    on="Job_Title", # Chave de Negócio para o JOIN
    how="inner"
)

# 4. Seleciona as colunas finais (FK + Métricas) para a Tabela Fato
df_fato_risco = df_fato_join.select(
    col("id_ocupacao"), # Chave Estrangeira
    col("Average_Salary"),
    col("Years_Experience"),
    col("AI_Exposure_Index"),
    col("Tech_Growth_Factor"),
    col("Automation_Probability_2030"),
    # As 10 Skills como Métricas
    col("Skill_1"), col("Skill_2"), col("Skill_3"), col("Skill_4"), col("Skill_5"),
    col("Skill_6"), col("Skill_7"), col("Skill_8"), col("Skill_9"), col("Skill_10")
)

# 5. Salva no Delta Lake na camada Curated
df_fato_risco.write.format("delta").mode("overwrite").saveAsTable("curated_data.fato_risco_automacao")

print("Tabela fato_risco_automacao criada com sucesso!")
# df_fato_risco.display()

In [0]:
from pyspark.sql.functions import corr, avg, col, lit

# Carrega o DataFrame da Tabela Fato
df_fato = spark.table(
    "curated_data.fato_risco_automacao"
)

In [0]:
# Carrega o DataFrame da Tabela Dimensão
df_dim = spark.table("curated_data.dim_ocupacao")

# Realiza o JOIN
df_join = df_fato.join(
    df_dim.select("id_ocupacao", "Job_Title"),
    on="id_ocupacao",
    how="inner"
)

# Agrupamento e cálculo da Média
top_10_risco = df_join.groupBy("Job_Title").agg(
    avg("Automation_Probability_2030").alias("Avg_Automation_Risk")
).orderBy(col("Avg_Automation_Risk").desc()).limit(10)

print("Top 10 Ocupações com Maior Risco de Automação:")
top_10_risco.show(truncate=False)

# Resultado esperado: AI Engineer no topo (se o resultado do seu dataset for mantido)

In [0]:
# Lista de todas as colunas de habilidades
skill_columns = [f"Skill_{i}" for i in range(1, 11)]

# DataFrame vazio para iniciar a união
df_union = None

# Itera sobre cada habilidade para calcular a correlação individual
for skill in skill_columns:
    df_corr_skill = df_fato.select(
        lit(skill).alias("Habilidade"),  # Adiciona o nome da skill como uma coluna literal
        corr("AI_Exposure_Index", skill).alias("Correlacao_Exposicao_IA")
    )
    
    # Inicia a união ou continua unindo os DataFrames
    if df_union is None:
        df_union = df_corr_skill
    else:
        df_union = df_union.unionAll(df_corr_skill)

# Ordena o DataFrame final pelo valor da correlação (da mais forte para a mais fraca)
resultado_correlacao_habilidades = df_union.orderBy(col("Correlacao_Exposicao_IA").desc())

print("Correlação Habilidade vs. Exposição à IA:")
resultado_correlacao_habilidades.show(truncate=False)

# Resultado esperado: Todos os valores muito próximos de zero.